# TAKE ONE RANDOM PAIR

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob
import matplotlib.patches as patches

In [ ]:
l = '/root/data/erko/single_pair/left_1533225206668.png'
r = '/root/data/erko/single_pair/right_1533225206668.png'

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(Image.open(l))
ax[1].imshow(Image.open(r))
plt.show()

In [ ]:
plt.imshow(np.array(Image.open(l))[0:700, 0:1500])
plt.show()

In [ ]:
plt.imshow(np.array(Image.open(r))[0:700, 0:1500])
plt.show()

In [ ]:
plt.imshow(np.array(Image.open(l))[820:1250, 1000:2700])
plt.show()

In [ ]:
plt.imshow(np.array(Image.open(r))[820:1250, 1000:2700])
plt.show()

In [ ]:
left_corners = np.array([[820, 1000], [1250, 1000], [1250, 2700], [820, 2700]])

In [ ]:
plt.imshow(np.array(Image.open(r))[880:1310, 1300:3000])
plt.show()

In [ ]:
right_corners = np.array([[880, 1300], [1310, 1300], [1310, 3000], [880, 3000]])

In [ ]:
left_corners

In [ ]:
right_corners

In [ ]:
left_center = np.mean(left_corners[:, [1, 0]][:2], axis=0)
right_center = np.mean(left_corners[:, [1, 0]][2:], axis=0)

In [ ]:
np.stack((left_center, right_center))

In [ ]:
# Create figure and axes
fig,ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(Image.open(l))
rect = patches.Polygon(left_corners[:, [1, 0]], closed=True, linewidth=2,edgecolor='r',facecolor='none')
ax[0].add_patch(rect)
ax[0].add_patch(patches.Circle(left_center, radius=10, facecolor='b'))
ax[0].add_patch(patches.Circle(right_center, radius=10, facecolor='b'))
ax[0].plot((left_center[0], right_center[0]), (left_center[1], right_center[1]), color='b')

ax[1].imshow(Image.open(r))
rect = patches.Polygon(right_corners[:, [1, 0]], closed=True, linewidth=2,edgecolor='r',facecolor='none')
ax[1].add_patch(rect)
plt.show()

In [ ]:
Image.open(l).size

In [ ]:
# estimate diparity
disp = 10
# fig,ax = plt.subplots(2, 1, figsize=(20, 10))
shifted = np.zeros((3000, 4096, 3))
shifted[:, disp:, :] = np.array(Image.open(l).convert('RGB'))[:, :-disp,:]
plt.imshow(shifted[820:1250, 1000:2700])

plt.imshow(np.array(Image.open(r))[880:1310, 1300:3000], alpha = 0.5)

In [ ]:
from PIL import Image, ImageDraw

In [ ]:
img = Image.new('L', (4096, 3000), 0)
ImageDraw.Draw(img).polygon([tuple(t) for t in list(left_corners[:, [1, 0]])], outline=1, fill=1)
mask = np.array(img)

In [ ]:
plt.imshow(mask)
plt.show()

In [ ]:
# disparities
disp = right_corners[:,1] - left_corners[:,1]
print(disp)

In [ ]:
# # calculate coordinates of central point
# left_center_left = np.mean(left_corners[1:3], axis=0)
# left_center_right = np.mean([left_corners[0], left_corners[-1]], axis=0)
# right_center_left = np.mean(right_corners[1:3], axis=0)
# right_center_right = np.mean([right_corners[0], right_corners[-1]], axis=0)

In [ ]:
# # disparities
# left_center_disp= (right_center_left - left_center_left)[1]
# right_center_disp= (right_center_right - left_center_right)[1]

In [ ]:
# plt.plot([left_center_left[1], left_center_right[1]], [left_center_disp, right_center_disp])
# plt.show()

In [ ]:
from scipy.interpolate import interp2d

In [ ]:
# interpolation 2d
f = interp2d(left_corners[:,1], left_corners[:,0], disp)

In [ ]:
left_corners[:,0]

In [ ]:
left_corners[:,1]

In [ ]:
disp

In [ ]:
ynew = np.arange(0, 3000, 1)
xnew = np.arange(0, 4096, 1)
znew = f(xnew, ynew)

In [ ]:
left_corners

In [ ]:
right_corners

In [ ]:
plt.imshow(znew)
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(np.array(Image.open(l)))
plt.imshow(znew*mask, alpha=0.7)
plt.colorbar()
plt.show()


# CONVERT TO DEPTH

In [ ]:
disparities = znew*mask

In [ ]:
# # meters
# focal_length = 10.0*1e-3
# baseline = 0.065
# image_sensor_width = 32.0*1e-3
# image_sensor_height = 18.0*1e-3
# pixel_size = image_sensor_width / 1280
# print(pixel_size)

In [ ]:
focal_length = 0.0107
baseline = 0.135
pixel_size_m = 3.45 * 1e-6 
focal_length_pixel = focal_length / pixel_size_m

In [ ]:
depth = focal_length_pixel*baseline / disparities

In [ ]:
depth

In [ ]:
plt.imshow(depth)
plt.colorbar()
plt.show()

In [ ]:
depth[2000, 2000]

In [ ]:
image_sensor_width = 0.01412
image_sensor_height = 0.01412

In [ ]:
def convert_to_world_point(x, y, depth_map):
    image_center_x = depth_map.shape[1] / 2.0
    image_center_y = depth_map.shape[0] / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (image_sensor_width / depth_map.shape[1])
    sensor_z = px_z * (image_sensor_height / depth_map.shape[0])
    
    d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
#     return world_y
    return (world_x, world_y, world_z)

In [ ]:
left_center

In [ ]:
right_center

In [ ]:
depth[1035, 1000]

In [ ]:
depth[1035, 2000]

In [ ]:
world_left = convert_to_world_point(1000, 1035, depth)

In [ ]:
world_right = convert_to_world_point(2700, 1035, depth)

In [ ]:
dist = np.linalg.norm(np.array(world_left) - np.array(world_right))

In [ ]:
print(dist)

In [ ]:
left_corners[:, [1, 0]]

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(20, 10))
ax.imshow(Image.open(l))
rect = patches.Polygon(left_corners[:, [1, 0]], closed=True, linewidth=2,edgecolor='r',facecolor='none')
ax.add_patch(rect)
ax.add_patch(patches.Circle(left_center, radius=10, facecolor='b'))
ax.add_patch(patches.Circle(right_center, radius=10, facecolor='b'))
ax.plot((left_center[0], right_center[0]), (left_center[1], right_center[1]), color='b')
ax.text(left_corners[:, [1, 0]][2,0], left_corners[:, [1, 0]][2,1]-10, str(dist)[:5] + 'm', color='b', size=15)
plt.show()